# Adquisición Anual

In [5]:
# Import Libraries
import ee
import h3
import geopandas as gpd
from tqdm.notebook import tqdm

# Initialize Earth Engine
def initialize_gee(project_id: str, opt_url: str = 'https://earthengine-highvolume.googleapis.com') -> None:
    """Initialize Google Earth Engine with authentication"""
    ee.Authenticate()
    ee.Initialize(project=project_id, opt_url=opt_url)

# Initialize with your project
initialize_gee('ee-nunezrimedio-tesina')  # Replace with your GEE project

# H3 to Earth Engine Conversion Functions
def get_h3_polygon_ee(h3_address):
    """
    Convert H3 address to Earth Engine geometry
    
    Parameters:
    -----------
    h3_address : str
        H3 hexagon address
        
    Returns:
    --------
    ee.Geometry.Polygon
        Earth Engine polygon geometry
    """
    # Get H3 hexagon boundary coordinates as [lat, lon]
    coords = h3.cell_to_boundary(h3_address)

    # Convert to [lon, lat] format required by Earth Engine
    coords_ee = [[lon, lat] for lat, lon in coords]

    # Create and return an Earth Engine polygon
    return ee.Geometry.Polygon([coords_ee])

def create_hexagon_feature(h3_address):
    """
    Create an Earth Engine Feature from an H3 address
    
    Parameters:
    -----------
    h3_address : str
        H3 hexagon address
        
    Returns:
    --------
    ee.Feature
        Earth Engine Feature with geometry and h3_address property
    """
    # Get hexagon geometry
    geometry = get_h3_polygon_ee(h3_address)

    # Create Feature with geometry and add h3_address as property
    return ee.Feature(geometry, {'h3_address': h3_address})

# Simplified NDVI Processing Function - Annual Median Only
def process_ndvi_annual_median(h3_addresses, output_folder='Lithium', batch_name='batch'):
    """
    Process annual median NDVI values for a list of H3 hexagons
    
    Parameters:
    -----------
    h3_addresses : list
        List of H3 hexagon addresses to analyze
    output_folder : str
        Google Drive folder for exporting results
    batch_name : str
        Name identifier for this batch
        
    Returns:
    --------
    ee.batch.Task
        Export task
    """
    # Create EE features from H3 addresses
    features = [create_hexagon_feature(h3_addr) for h3_addr in h3_addresses]
    hexagons = ee.FeatureCollection(features)

    # Get NDVI collection from MODIS
    ndvi_collection = ee.ImageCollection("MODIS/MCD43A4_006_NDVI").select('NDVI')

    def analyze_hexagon(feature):
        geom = feature.geometry()

        # Calculate annual median NDVI for each year
        years = ee.List.sequence(2000, 2022)

        def process_year(year):
            year = ee.Number(year)
            # Convert to integer format without decimal point
            year_str = ee.Number(year).int().format()

            # Define year date range
            year_start = ee.Date.fromYMD(year, 1, 1)
            year_end = ee.Date.fromYMD(year.add(1), 1, 1)

            # Filter NDVI collection for the year
            annual_collection = ndvi_collection.filterDate(year_start, year_end)

            # Calculate median NDVI for the year
            median_ndvi = annual_collection.median()

            # Calculate median value for the hexagon
            annual_median = median_ndvi.reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=geom,
                scale=500,  # MODIS resolution
                bestEffort=True
            )

            # Extract median value or use -9999 if no data
            median_value = ee.Number(ee.Dictionary(annual_median).get('NDVI', -9999))

            # Create property name for this year
            median_key = ee.String('annual_median_').cat(year_str)

            # Return dictionary with year's median
            return ee.Dictionary.fromLists([median_key], [median_value])

        # Process all years and combine results
        yearly_medians = ee.Dictionary(years.map(process_year).iterate(
            lambda dict_year, result: ee.Dictionary(result).combine(ee.Dictionary(dict_year), False),
            ee.Dictionary({})
        ))

        # Add yearly medians as properties to the feature
        return feature.set(yearly_medians)

    # Process all hexagons
    processed_hexagons = hexagons.map(analyze_hexagon)

    # Export to Drive - using a single folder level
    task = ee.batch.Export.table.toDrive(
        collection=processed_hexagons,
        description=f'NDVI_Annual_Median_{batch_name}',
        folder='Lithium_NDVI',  # Single folder name without path separators
        fileFormat='GeoJSON'
    )

    task.start()
    print(f"Task started: NDVI_Annual_Median_{batch_name}")
    return task

# Batch Processing Function
def process_in_batches(h3_addresses, batch_size=500, output_folder='Lithium'):
    """
    Process hexagons in batches to avoid GEE limitations
    
    Parameters:
    -----------
    h3_addresses : list
        List of H3 addresses to analyze
    batch_size : int
        Number of hexagons per batch
    output_folder : str
        Google Drive folder for exporting results
        
    Returns:
    --------
    list
        List of export tasks
    """
    tasks = []
    total_hexagons = len(h3_addresses)

    for i in range(0, total_hexagons, batch_size):
        batch = h3_addresses[i:i+batch_size]
        batch_num = i//batch_size + 1
        total_batches = (total_hexagons+batch_size-1)//batch_size

        print(f"Processing batch {batch_num}/{total_batches} ({len(batch)} hexagons)")
        task = process_ndvi_annual_median(
            batch,
            output_folder=output_folder,
            batch_name=f'batch_{batch_num}_of_{total_batches}'
        )
        tasks.append(task)

    return tasks

# Load Hexagons and Run Analysis
# Load the hexagons file
hexagon_file_path = "/Users/santi/DataspellProjects/Lithium-Jakob/Data/lithium_hexagons_res6.geojson"
gdf = gpd.read_file(hexagon_file_path)
print(f"Loaded {len(gdf)} hexagons")

# Extract H3 addresses
if 'h3_index' in gdf.columns:
    h3_addresses = gdf["h3_index"].to_list()
else:
    h3_addresses = gdf.index.to_list()  # If H3 indices are in the GeoDataFrame index

# Define output folder - single folder name
output_folder = 'Lithium_NDVI'

# Process in batches
tasks = process_in_batches(
    h3_addresses=h3_addresses,
    batch_size=1000,  # Reduced batch size for better reliability
    output_folder=output_folder
)

print(f"Started {len(tasks)} export tasks. Monitor progress in the Earth Engine Code Editor.")

Loaded 640 hexagons
Processing batch 1/1 (640 hexagons)
Task started: NDVI_Annual_Median_batch_1_of_1
Started 1 export tasks. Monitor progress in the Earth Engine Code Editor.


In [2]:
import geopandas as gpd
gpd.read_file("/Users/santi/DataspellProjects/Lithium-Jakob/Data/NDVI_Annual_Median_batch_1_of_1.geojson")

,id,annual_median_2000,annual_median_2001,annual_median_2002,annual_median_2003,annual_median_2004,annual_median_2005,annual_median_2006,annual_median_2007,annual_median_2008,...,annual_median_2015,annual_median_2016,annual_median_2017,annual_median_2018,annual_median_2019,annual_median_2020,annual_median_2021,annual_median_2022,h3_address,geometry
0,0,0.066153,0.072029,0.062791,0.061377,0.060942,0.074544,0.066704,0.065670,0.069492,...,0.072089,0.064134,0.069337,0.076888,0.076033,0.084484,0.071358,0.085036,86b3506b7ffffff,"POLYGON ((-66.87866 -23.99008, -66.90726 -24.0..."
1,1,0.130558,0.131119,0.134647,0.129128,0.134161,0.133558,0.134018,0.128546,0.132286,...,0.137495,0.133521,0.135903,0.139375,0.137858,0.139963,0.135926,0.138921,86b350627ffffff,"POLYGON ((-66.66966 -23.90124, -66.69817 -23.9..."
2,2,0.075935,0.079314,0.078257,0.070437,0.070647,0.081323,0.077702,0.071132,0.076326,...,0.082855,0.075581,0.077902,0.082222,0.082158,0.086158,0.076885,0.087252,86b3507a7ffffff,"POLYGON ((-66.80663 -23.82564, -66.83519 -23.8..."
3,3,0.082133,0.086781,0.084274,0.081210,0.081181,0.078784,0.080958,0.079465,0.084881,...,0.088081,0.085307,0.089088,0.091077,0.092982,0.099254,0.093548,0.097282,86b35042fffffff,"POLYGON ((-66.89567 -23.63998, -66.92424 -23.6..."
4,4,0.096824,0.100180,0.101294,0.102200,0.103171,0.102690,0.104155,0.103303,0.101367,...,0.103347,0.105033,0.104104,0.105809,0.106324,0.107780,0.106258,0.102180,86b3539afffffff,"POLYGON ((-66.50526 -24.17251, -66.53374 -24.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635,0.106679,0.118734,0.110508,0.096334,0.111926,0.110138,0.106504,0.109147,0.102334,...,0.117865,0.102179,0.106654,0.112156,0.106269,0.118550,0.102157,0.120208,86b35149fffffff,"POLYGON ((-66.17039 -24.31013, -66.19876 -24.3..."
636,636,0.085739,0.087500,0.088509,0.084577,0.089438,0.083372,0.084132,0.083855,0.085132,...,0.092463,0.088113,0.091906,0.097335,0.089233,0.090302,0.090456,0.092593,86b35568fffffff,"POLYGON ((-65.77841 -23.53271, -65.80656 -23.5..."
637,637,0.092190,0.095816,0.092910,0.081981,0.094136,0.095082,0.095554,0.084326,0.088042,...,0.101430,0.088074,0.088034,0.095828,0.092113,0.101881,0.097499,0.100505,86b35022fffffff,"POLYGON ((-66.20200 -23.86440, -66.23034 -23.8..."
638,638,-0.003250,0.000837,0.004614,0.010116,0.013405,0.003389,0.004204,0.006115,0.002514,...,-0.002734,0.019172,0.012256,0.010300,0.017112,0.009183,0.005248,0.002597,86b351db7ffffff,"POLYGON ((-66.00858 -23.67702, -66.03684 -23.6..."


# Adquisición 3 meses

In [8]:
# Import Libraries
import ee
import h3
import geopandas as gpd
from tqdm.notebook import tqdm

# Initialize Earth Engine
def initialize_gee(project_id: str, opt_url: str = 'https://earthengine-highvolume.googleapis.com') -> None:
    """Initialize Google Earth Engine with authentication"""
    ee.Authenticate()
    ee.Initialize(project=project_id, opt_url=opt_url)

# Initialize with your project
initialize_gee('ee-nunezrimedio-tesina')  # Replace with your GEE project

# H3 to Earth Engine Conversion Functions
def get_h3_polygon_ee(h3_address):
    """
    Convert H3 address to Earth Engine geometry
    
    Parameters:
    -----------
    h3_address : str
        H3 hexagon address
        
    Returns:
    --------
    ee.Geometry.Polygon
        Earth Engine polygon geometry
    """
    # Get H3 hexagon boundary coordinates as [lat, lon]
    coords = h3.cell_to_boundary(h3_address)

    # Convert to [lon, lat] format required by Earth Engine
    coords_ee = [[lon, lat] for lat, lon in coords]

    # Create and return an Earth Engine polygon
    return ee.Geometry.Polygon([coords_ee])

def create_hexagon_feature(h3_address):
    """
    Create an Earth Engine Feature from an H3 address
    
    Parameters:
    -----------
    h3_address : str
        H3 hexagon address
        
    Returns:
    --------
    ee.Feature
        Earth Engine Feature with geometry and h3_address property
    """
    # Get hexagon geometry
    geometry = get_h3_polygon_ee(h3_address)

    # Create Feature with geometry and add h3_address as property
    return ee.Feature(geometry, {'h3_address': h3_address})

# Quarterly NDVI Processing Function - Fixed for EE compatibility
def process_ndvi_quarterly(h3_addresses, output_folder='Lithium', batch_name='batch'):
    """
    Process quarterly NDVI values for a list of H3 hexagons
    
    Parameters:
    -----------
    h3_addresses : list
        List of H3 hexagon addresses to analyze
    output_folder : str
        Google Drive folder for exporting results
    batch_name : str
        Name identifier for this batch
        
    Returns:
    --------
    ee.batch.Task
        Export task
    """
    # Create EE features from H3 addresses
    features = [create_hexagon_feature(h3_addr) for h3_addr in h3_addresses]
    hexagons = ee.FeatureCollection(features)

    # Get NDVI collection from MODIS
    ndvi_collection = ee.ImageCollection("MODIS/MCD43A4_006_NDVI").select('NDVI')

    def analyze_hexagon(feature):
        geom = feature.geometry()

        # Calculate quarterly NDVI for each year
        years = ee.List.sequence(2000, 2022)

        def process_year(year):
            year = ee.Number(year)
            # Convert to integer format without decimal point
            year_str = ee.Number(year).int().format()

            # Define the quarters as individual functions rather than a list of dictionaries
            # Southern Hemisphere seasons

            # Q1 - Summer (Jan-Mar)
            def process_q1():
                quarter_start = ee.Date.fromYMD(year, 1, 1)
                quarter_end = ee.Date.fromYMD(year, 4, 1)

                # Filter NDVI collection for the quarter
                quarter_collection = ndvi_collection.filterDate(quarter_start, quarter_end)

                # Calculate median NDVI for the quarter
                median_ndvi = quarter_collection.median()

                # Calculate median value for the hexagon
                quarter_median = median_ndvi.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom,
                    scale=500,  # MODIS resolution
                    bestEffort=True
                )

                # Extract median value or use -9999 if no data
                median_value = ee.Number(ee.Dictionary(quarter_median).get('NDVI', -9999))

                # Create property name for this quarter
                median_key = ee.String('Q1_Summer_').cat(year_str)

                # Return dictionary with quarter's median
                return ee.Dictionary.fromLists([median_key], [median_value])

            # Q2 - Fall (Apr-Jun)
            def process_q2():
                quarter_start = ee.Date.fromYMD(year, 4, 1)
                quarter_end = ee.Date.fromYMD(year, 7, 1)

                quarter_collection = ndvi_collection.filterDate(quarter_start, quarter_end)
                median_ndvi = quarter_collection.median()

                quarter_median = median_ndvi.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom,
                    scale=500,
                    bestEffort=True
                )

                median_value = ee.Number(ee.Dictionary(quarter_median).get('NDVI', -9999))
                median_key = ee.String('Q2_Fall_').cat(year_str)

                return ee.Dictionary.fromLists([median_key], [median_value])

            # Q3 - Winter (Jul-Sep)
            def process_q3():
                quarter_start = ee.Date.fromYMD(year, 7, 1)
                quarter_end = ee.Date.fromYMD(year, 10, 1)

                quarter_collection = ndvi_collection.filterDate(quarter_start, quarter_end)
                median_ndvi = quarter_collection.median()

                quarter_median = median_ndvi.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom,
                    scale=500,
                    bestEffort=True
                )

                median_value = ee.Number(ee.Dictionary(quarter_median).get('NDVI', -9999))
                median_key = ee.String('Q3_Winter_').cat(year_str)

                return ee.Dictionary.fromLists([median_key], [median_value])

            # Q4 - Spring (Oct-Dec)
            def process_q4():
                quarter_start = ee.Date.fromYMD(year, 10, 1)
                quarter_end = ee.Date.fromYMD(year.add(1), 1, 1)

                quarter_collection = ndvi_collection.filterDate(quarter_start, quarter_end)
                median_ndvi = quarter_collection.median()

                quarter_median = median_ndvi.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom,
                    scale=500,
                    bestEffort=True
                )

                median_value = ee.Number(ee.Dictionary(quarter_median).get('NDVI', -9999))
                median_key = ee.String('Q4_Spring_').cat(year_str)

                return ee.Dictionary.fromLists([median_key], [median_value])

            # Process each quarter and combine results
            q1_results = process_q1()
            q2_results = process_q2()
            q3_results = process_q3()
            q4_results = process_q4()

            # Combine all quarters for this year
            return q1_results.combine(q2_results).combine(q3_results).combine(q4_results)

        # Process all years and combine results
        all_results = ee.Dictionary(years.map(process_year).iterate(
            lambda dict_year, result: ee.Dictionary(result).combine(ee.Dictionary(dict_year), False),
            ee.Dictionary({})
        ))

        # Add results as properties to the feature
        return feature.set(all_results)

    # Process all hexagons
    processed_hexagons = hexagons.map(analyze_hexagon)

    # Export to Drive - using a single folder level
    task = ee.batch.Export.table.toDrive(
        collection=processed_hexagons,
        description=f'NDVI_Quarterly_{batch_name}',
        folder='Lithium_NDVI',  # Use the same folder as the original code
        fileFormat='GeoJSON'
    )

    task = ee.batch.Export.table.toDrive(
        collection=processed_hexagons,
        description=f'batch_{batch_name}',  # Cambiado
        folder='Lithium_NDVI',              # Única carpeta
        fileFormat='GeoJSON'
    )

    task.start()
    print(f"Task started: NDVI_Quarterly_{batch_name}")
    return task

# Batch Processing Function for Quarterly Analysis
def process_quarterly_in_batches(h3_addresses, batch_size=500, output_folder='Lithium'):
    """
    Process hexagons in batches for quarterly NDVI analysis
    
    Parameters:
    -----------
    h3_addresses : list
        List of H3 addresses to analyze
    batch_size : int
        Number of hexagons per batch
    output_folder : str
        Google Drive folder for exporting results
        
    Returns:
    --------
    list
        List of export tasks
    """
    tasks = []
    total_hexagons = len(h3_addresses)

    for i in range(0, total_hexagons, batch_size):
        batch = h3_addresses[i:i+batch_size]
        batch_num = i//batch_size + 1
        total_batches = (total_hexagons+batch_size-1)//batch_size

        print(f"Processing batch {batch_num}/{total_batches} ({len(batch)} hexagons)")
        task = process_ndvi_quarterly(
            batch,
            output_folder=output_folder,
            batch_name=f'batch_{batch_num}_of_{total_batches}'
        )
        tasks.append(task)

    return tasks

# Load Hexagons and Run Analysis
# Load the hexagons file
hexagon_file_path = "/Users/santi/DataspellProjects/Lithium-Jakob/Data/lithium_hexagons_res6.geojson"
gdf = gpd.read_file(hexagon_file_path)
print(f"Loaded {len(gdf)} hexagons")

# Extract H3 addresses
if 'h3_index' in gdf.columns:
    h3_addresses = gdf["h3_index"].to_list()
else:
    h3_addresses = gdf.index.to_list()  # If H3 indices are in the GeoDataFrame index

# Define output folder - single folder name
output_folder = 'Lithium_NDVI'  # Use the same folder as the original code

# Process in batches
tasks = process_quarterly_in_batches(
    h3_addresses=h3_addresses,
    batch_size=25,  # Reduced batch size for better reliability
    output_folder=output_folder
)


print(f"Started {len(tasks)} export tasks. Monitor progress in the Earth Engine Code Editor.")

Loaded 640 hexagons
Processing batch 1/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_1_of_26
Processing batch 2/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_2_of_26
Processing batch 3/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_3_of_26
Processing batch 4/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_4_of_26
Processing batch 5/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_5_of_26
Processing batch 6/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_6_of_26
Processing batch 7/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_7_of_26
Processing batch 8/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_8_of_26
Processing batch 9/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_9_of_26
Processing batch 10/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_10_of_26
Processing batch 11/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_11_of_26
Processing batch 12/26 (25 hexagons)
Task started: NDVI_Quarterly_batch_12_of_26
Processing batch 13/26 (25